# PHYS 304 Computing Project - Relaxation Method
### Name: Jacob Buchanan

Consider a closed metal box having a rectangular cross section $a \times b \times c$. The lid (in figure) is located in the $xy$-plane at $z=c$ and is maintained at a potential $V_0$ relative to the other five sides of the box which are connected to the ground. Imagine the lid is electrically isolated from the rest of the box.

The full potential is given by,

$V(x,y,z)=\frac{16V_0}{\pi^2}\sum_{m,n=odd}^{\infty}\frac{1}{mn}\sin{\frac{m\pi x}{a}}\sin{\frac{n\pi y}{b}}\frac{\sinh{\gamma_{mn}z}}{\sinh{\gamma_{mn}c}}$

where $m$ and $n$ are odd integers and $\gamma_{mn}=\pi \sqrt{\frac{m^2}{a^2}+\frac{n^2}{b^2}}$.

Approximate solutions for $V(x,y,z)$ may be obtained by truncating the infinite series at specified upper limits for both of the summations in the expression for V above.

Let $a=b=c=10cm$ and $V_0=1000V$. Let the tolerance between iterations be $1\times 10^{-6}$.

Now calculate the potential at discrete points inside the box on a lattice having unit cell size $\Delta x=\Delta y=\Delta z=1cm$. For this purpose, take $1\leq x \leq 9cm$, $1\leq y \leq 9cm$, and $0\leq z \leq 9cm$. Produce a grid displaying the potentials (to one decimal place) at twenty evenly space points in the $yz$-plane for $x=5cm$.

## Solution

In [2]:
# Import the necessities
import numpy as np
import matplotlib.pyplot as plt